# **Clasificador de Imágenes de Flores**

## Librerías.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import skimage as ski
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, make_scorer

## Importación del Dataset

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        directory = '/content/drive/My Drive/ProyectoIA/flowers',
        target_size=(100, 100),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        directory = '/content/drive/My Drive/ProyectoIA/flowers',
        target_size=(100, 100),
        batch_size=32,
        class_mode='binary')

Found 11792 images belonging to 16 classes.
Found 11792 images belonging to 16 classes.


**Verificación del archivo.**

In [5]:
base_dir =  r'/content/drive/My Drive/ProyectoIA/flowers'
folders = os.listdir(base_dir)
print(folders)

['california_poppy', 'dandelion', 'coreopsis', 'daffodil', 'common_daisy', 'carnation', 'astilbe', 'black_eyed_susan', 'calendula', 'bellflower', 'tulip', 'magnolia', 'water_lily', 'sunflower', 'rose', 'iris']


## Cargar imágenes a cada categoría.

In [6]:
# Listas de nombres de archivo para cada categoría
water_lily = sorted(os.listdir(os.path.join(base_dir, 'water_lily')))
tulip = sorted(os.listdir(os.path.join(base_dir, 'tulip')))
sunflower = sorted(os.listdir(os.path.join(base_dir, 'sunflower')))
rose = sorted(os.listdir(os.path.join(base_dir, 'rose')))
magnolia = sorted(os.listdir(os.path.join(base_dir, 'magnolia')))
iris = sorted(os.listdir(os.path.join(base_dir, 'iris')))
dandelion = sorted(os.listdir(os.path.join(base_dir, 'dandelion')))
daffodil = sorted(os.listdir(os.path.join(base_dir, 'daffodil')))
coreopsis = sorted(os.listdir(os.path.join(base_dir, 'coreopsis')))
common_daisy = sorted(os.listdir(os.path.join(base_dir, 'common_daisy')))
carnation = sorted(os.listdir(os.path.join(base_dir, 'carnation')))
california_poppy = sorted(os.listdir(os.path.join(base_dir, 'california_poppy')))
calendula = sorted(os.listdir(os.path.join(base_dir, 'calendula')))
black_eyed_susan = sorted(os.listdir(os.path.join(base_dir, 'black_eyed_susan')))
bellflower = sorted(os.listdir(os.path.join(base_dir, 'bellflower')))
astilbe = sorted(os.listdir(os.path.join(base_dir, 'astilbe')))

## Redimensión y tratamiento de imágenes

In [11]:
data = []  # X
labels = []  # y

# Categorías y sus correspondientes valores de etiqueta
categorias = {
    'california_poppy': 0,
    'dandelion': 1,
    'coreopsis': 2,
    'daffodil': 3,
    'common_daisy': 4,
    'carnation': 5,
    'astilbe': 6,
    'black_eyed_susan': 7,
    'calendula': 8,
    'bellflower': 9,
    'tulip': 10,
    'magnolia': 11,
    'water_lily': 12,
    'sunflower': 13,
    'rose': 14,
    'iris': 15
}

# Iteración sobre las categorías y sus imágenes
for categoria, label in categorias.items():
    carpeta = base_dir + '/' + categoria + '/'
    for imagen in os.listdir(carpeta):
        img = cv2.imread(carpeta + imagen)
        gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gris = cv2.resize(gris, (64, 64))
        data.append(gris.flatten())
        labels.append(label)

## Permutación de filas y particionado


In [12]:
imagenes = np.array(data)
labels = np.array(labels)
#Ajuste al formato de permutation
imagenes = imagenes.astype(np.float32)
labels = labels.astype(np.int32)
imagenes = imagenes/255 #Normalización de los valores por píxel (0 a 1). Negro=0, Blanco=255

#Permutación
p = np.random.permutation(range(len(imagenes)))
imagenes = imagenes[p]
labels = labels[p]

Partición 80/20

In [13]:
X_train, X_test, y_train, y_test = train_test_split(imagenes, labels, test_size = 0.2, random_state = 31)

## Desicion Tree

In [15]:
est = DecisionTreeClassifier()
est.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = est.predict(X_test)

# Calculo del accuracy
accuracy = accuracy_score(y_test, predicciones)
print("Accuracy score: %.3f" % accuracy)

Accuracy score: 0.132


In [23]:
est = DecisionTreeClassifier(max_depth=25, criterion='gini')
est.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = est.predict(X_test)

# Calculo el accuracy score
accuracy = accuracy_score(y_test, predicciones)
print("Accuracy score: %.3f" % accuracy)

Accuracy score: 0.141


## Random Forest

In [ ]:
est2 = RandomForestClassifier()
est2.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = est2.predict(X_test)

# Calculo del accuracy
accuracy = accuracy_score(y_test, predicciones)
print("Accuracy score: %.3f" % accuracy)

Accuracy score: 0.245


In [16]:
est2 = RandomForestClassifier(n_estimators=200, max_depth=30, criterion='entropy')
est2.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = est2.predict(X_test)

# Calculo del accuracy
accuracy = accuracy_score(y_test, predicciones)
print("Accuracy score: %.3f" % accuracy)

Accuracy score: 0.254


## SVC

In [17]:
est3 = SVC()
est3.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = est3.predict(X_test)

# Calculo del accuracy
accuracy = accuracy_score(y_test, predicciones)
print("Accuracy score: %.3f" % accuracy)

KeyboardInterrupt: 

In [ ]:
est3 = SVC(kernel='linear')
est3.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = est3.predict(X_test)

# Calcular el accuracy score
accuracy = accuracy_score(y_test, predicciones)
print("Accuracy score: %.3f" % accuracy)